In [46]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import random

#https://www.tripadvisor.com/Restaurants-g293986-Amman_Amman_Governorate.html
"""
soup.find_all('a', {'class':'property_title'})
soup.find_all('span', {'class':'reviewCount'})
soup.find_all('div', {'class':'popIndex rebrand popIndexDefault'}) ranking
soup.find_all('span', {'class':'item price'})
soup.find_all('a', {'class':'item cuisine'})

"""
                          
property_titles = []
review_count = []
rankings = []
prices = []
cuisines = []
url2 = []

for offset in range(0, 19):
    url = 'https://www.tripadvisor.com/Restaurants-g293986-oa' + str(offset) + '-Amman_Amman_Governorate.html#EATERY_OVERVIEW_BOX'

    r = requests.get(url)
#    soup = BeautifulSoup(r.text, "html.parser")
    time.sleep(0.05)
    soup = BeautifulSoup(r.text,"lxml")
#    li = soup.find_all('div', {'id':'EATERY_SEARCH_RESULTS'}) # all restaurants in listing
    
#    for restaurant in soup.find_all(')
    for link in soup.find_all('a', {'class':'property_title'}):
        title = link.text
        property_titles.append(title.strip('\n'))
        
    for link in soup.find_all('span', {'class':'reviewCount'}):
        count = link.text
        review_count.append(count.strip('\n'))
    
    for link in soup.find_all('div', {'class':'popIndex rebrand popIndexDefault'}):
        rank = link.text
        rankings.append(rank.strip('\n'))
        
    # some restaurants don't have a price, have to figure this out
    for link in soup.find_all('span', {'class':'item price'}):
        price = link.text
        prices.append(price.strip('\n'))
       
    # this isn't really working as of now -- it collects all cuisine tags, not just one for each restaurant
    for link in soup.find_all('a', {'class':'item cuisine'}):
        cuisine = link.text
        cuisines.append(cuisine.strip('\n'))
    
    #added url so that we can use these links to move through all the pages in tripadvisor_restaurant.py 
    # "https://tripadvisor.com/ + url_of_rest
    for link in soup.find_all('a',{'class':'property_title'}):
        url_of_rest = link['href']
        url2.append(url_of_rest.strip('\n'))

#columns = {'property_title':property_titles, 'review_count':review_count, 'ranking':rankings}
columns = {'property_title':property_titles, 'review_count':review_count, 'url2':url2, 'ranking':rankings}
df = pd.DataFrame(columns)
print(df[:10])

              property_title                          ranking    review_count  \
0                    Brisket   #1 of 545 Restaurants in Amman    268 reviews    
1            Pizza Roma Cafe   #2 of 545 Restaurants in Amman    300 reviews    
2             Habibah Sweets   #3 of 545 Restaurants in Amman    665 reviews    
3      Fakhreldin Restaurant   #4 of 545 Restaurants in Amman    787 reviews    
4           Sufra Restaurant   #5 of 545 Restaurants in Amman    700 reviews    
5      V Lounge & Restaurant   #6 of 545 Restaurants in Amman     57 reviews    
6           Tawaheen al-Hawa   #7 of 545 Restaurants in Amman    713 reviews    
7  Chestnut Restaurant & Pub   #8 of 545 Restaurants in Amman    283 reviews    
8                     Hashem   #9 of 545 Restaurants in Amman  1,361 reviews    
9                  Abu Jbara  #10 of 545 Restaurants in Amman    348 reviews    

                                                url2  
0  /Restaurant_Review-g293986-d7267482-Reviews-Br... 